[View in Colaboratory](https://colab.research.google.com/github/zhangjunli177/tensorflow/blob/master/age_pred.ipynb)

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
#!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
#!apt-get update -qq 2>&1 > /dev/null
#!apt-get -y install -qq google-drive-ocamlfuse fuse
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [2]:
!pwd

/content


In [3]:
!ls

adc.json					 sample_data
google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb  wget-log


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!mkdir -p multi-tag

In [13]:
cd age_pred

/content/drive/DNN/age_pred


In [3]:
ls

adc.json  google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb  sample_data/
drive/    multi-tag/                                       wget-log


In [0]:
!cp /content/drive/DNN/age_pred/* /content/multi-tag/

In [7]:
cd /content/multi-tag

/content/multi-tag


In [8]:
!ls -l

total 477360
-rw-r--r-- 1 root root 480000096 Oct  9 12:07 new_feature_1200.npy
-rw-r--r-- 1 root root   8800080 Oct  9 12:07 true_label.npy
-rw-r--r-- 1 root root      5147 Oct  9 12:07 zjl_no_noise_1200.ipynb


In [9]:
!md5sum new_feature_1200.npy

8310acbe917eea4eb8b233ac6f0f4eca  new_feature_1200.npy


In [10]:
!md5sum true_label.npy

c77d742f75da629fada41e2989e29867  true_label.npy


In [11]:
#add gauss noise layer
import numpy as np
from keras.layers import Dense,Flatten,Input,Dropout,GaussianNoise
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPool1D
from keras.layers.merge import concatenate
from keras.models import Model
from keras.models import load_model
from keras.optimizers import Adam
import keras
import math
import os
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from keras import backend as k_backend
from keras import regularizers
from sklearn import preprocessing
from sklearn.decomposition import PCA
from keras.layers import BatchNormalization


def my_logloss(y_true,y_pred):
    return -k_backend.sum(y_true*k_backend.log(y_pred),axis=-1)

#train_data_634=np.load('feature_634_gyh_lxtz.npy')
train_data_1200=np.load('new_feature_1200.npy')
data_len=len(train_data_1200)
#加噪声
noise_data_coef =0.05*np.random.randn(train_data_1200.shape[0],1,
                         train_data_1200.shape[2])
train_data_1200_noise=(1+noise_data_coef)*train_data_1200
#
train_data_1200=np.concatenate((train_data_1200,train_data_1200_noise),axis=0)
#去均值
train_data_1200=train_data_1200-train_data_1200.mean(axis=0)


#train_data_1200=train_data_1200.reshape(data_len,1200)
#new_feature_1052.npy
#scaler = preprocessing.StandardScaler().fit(train_data_1200)
#train_data_1200=scaler.transform(train_data_1200) 
#pca 降维
#pcaClf = PCA(n_components=0.99, whiten=True)
#train_data_1200=pcaClf.fit_transform(train_data_1200)
#train_data_1200=train_data_1200.reshape(data_len,1,train_data_1200.shape[1])
#train_data_1052=train_data_1052.reshape(data_len,1,1052)
true_label=np.load('true_label.npy')
true_label=true_label.reshape(data_len,1,22)
true_label=np.concatenate((true_label,true_label),axis=0)
validation_split=0.1
batch_size=1024*8
epochs=200
num_classes=22

dropout_rate=0.5
dropout_rate_model=0.5
l2_rate=0.001
#haha，pencil model
input_data=Input(shape=(1,train_data_1200.shape[2]),dtype='float')

x1=Dense(1200,activation='relu',
        kernel_regularizer=regularizers.l2(l2_rate))(input_data)
x2=concatenate([input_data,x1],axis=2)
x2=GaussianNoise(0.05)(x2)
x3=Dropout(dropout_rate)(x2)
x3=BatchNormalization()(x3)
x4=Dense(600,activation='relu',
        kernel_regularizer=regularizers.l2(l2_rate))(x3)
x5=concatenate([x2,x4],axis=2)
x5=GaussianNoise(0.05)(x5)
x6=Dropout(dropout_rate)(x5)
x6=BatchNormalization()(x6)
x7=Dense(300,activation='relu',
       kernel_regularizer=regularizers.l2(l2_rate))(x6)
x8=concatenate([x5,x7],axis=2)
x8=GaussianNoise(0.05)(x8)
x9=Dropout(dropout_rate)(x8)
x9=BatchNormalization()(x9)
x10=Dense(150,activation='relu',
       kernel_regularizer=regularizers.l2(l2_rate))(x9)
x11=concatenate([x8,x10],axis=2)
x11=GaussianNoise(0.05)(x11)
x12=Dropout(dropout_rate)(x11)
x12=BatchNormalization()(x12)
x13=Dense(80,activation='relu',
       kernel_regularizer=regularizers.l2(l2_rate))(x12)
x14=concatenate([x11,x13],axis=2)
x14=GaussianNoise(0.05)(x14)
x15=Dropout(dropout_rate_model)(x14)
x15=BatchNormalization()(x15)
outputs=Dense(num_classes,activation='softmax')(x15)
model=Model(input_data,outputs)

print('model generated')

def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 80:
        lr *=0.5*0.5*0.5*0.5*0.5
    if (epoch > 50) and (epoch < 80):
        lr *=0.5*0.5*0.5*0.5
    if (epoch > 30) and (epoch < 50):
        lr *= 0.5*0.5*0.5
    elif (epoch > 20) and (epoch <=30):
        lr *= 0.5*0.5
    elif (epoch > 5) and (epoch <=20):
        lr *= 0.5
    elif (epoch > 2) and (epoch <=5):
        lr *= 0.8
    print('Learning rate: ', lr)
    return lr

print('begin training')
X_train,X_test, y_train, y_test =train_test_split(train_data_1200,true_label,test_size=validation_split, random_state=0)
print('data splited')

model.compile(loss='categorical_crossentropy',
              #loss=my_logloss,
              optimizer=Adam(0.001),#(lr=lr_schedule(0)),#(lr=l0.001),
              metrics=['accuracy'])
model.summary()
model_type='DNN_1200'
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'male_age_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             #monitor='val_acc',
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=1e-6)

#callbacks = [checkpoint, lr_reducer,lr_scheduler]
callbacks = [checkpoint, lr_reducer]

train = model.fit(X_train,y_train,epochs=epochs,
                  batch_size=batch_size,validation_data=(X_test,y_test),
                  verbose=1, callbacks=callbacks)
scores=model.evaluate(X_test,y_test,batch_size=batch_size,verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

y_pred_dnn22=model.predict(X_test)
y_pred_dnn22=y_pred_dnn22.reshape(len(y_pred_dnn22),22)
y_test=y_test.reshape(len(y_test),22)
result=0
log_loss_score=0
for i in range(len(y_pred_dnn22)):
    for j in range(22):
        yp=y_pred_dnn22[i][j]
        if yp==0.0:
            yp=1e-6
        result+=(math.log(yp))*(y_test[i][j])
    log_loss_score+=result
    result=0
log_loss_score=-log_loss_score/len(y_pred_dnn22)
print('log_loss_score=%f' %log_loss_score)


Using TensorFlow backend.


model generated
begin training
data splited
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1, 1200)      0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1, 1200)      1441200     input_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 1, 2400)      0           input_1[0][0]                    
                                                                 dense_1[0][0]                    
__________________________________________________________________________________________________
gaussian_noise_1 (GaussianNoise (None, 1, 2400)      0           

In [13]:
import numpy as np
from keras.layers import Dense,Flatten,Input,Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPool1D
from keras.layers.merge import concatenate
from keras.models import Model
from keras.models import load_model
from keras.optimizers import Adam
import keras
import math
import os
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from keras import backend as k_backend
from keras import regularizers
from sklearn import preprocessing
from sklearn.decomposition import PCA
from keras.layers import BatchNormalization


def my_logloss(y_true,y_pred):
    return -k_backend.sum(y_true*k_backend.log(y_pred),axis=-1)
train_data_1200=np.load('new_feature_1200.npy')
train_data_1200=train_data_1200-train_data_1200.mean()
data_len=len(train_data_1200)
true_label=np.load('true_label.npy')
true_label=true_label.reshape(data_len,1,22)
validation_split=0.1
batch_size=2048*8
epochs=1000
num_classes=22

#dropout_rate=0.8
#dropout_rate_model=0.8
#l2_rate=0.0001
dropout_rate=0.8
dropout_rate_model=0.8
l2_rate=0.01

#haha，pencil model
input_data=Input(shape=(1,train_data_1200.shape[2]),dtype='float')
x1=Dense(1200,activation='relu',
        kernel_regularizer=regularizers.l2(l2_rate))(input_data)
x2=concatenate([input_data,x1],axis=2)
x3=Dropout(dropout_rate)(x2)
x3=BatchNormalization()(x3)
x4=Dense(600,activation='relu',
        kernel_regularizer=regularizers.l2(l2_rate))(x3)
x5=concatenate([x2,x4],axis=2)
x6=Dropout(dropout_rate)(x5)
x6=BatchNormalization()(x6)
x7=Dense(300,activation='relu',
       kernel_regularizer=regularizers.l2(l2_rate))(x6)
x8=concatenate([x5,x7],axis=2)
x9=Dropout(dropout_rate)(x8)
x9=BatchNormalization()(x9)
x10=Dense(150,activation='relu',
       kernel_regularizer=regularizers.l2(l2_rate))(x9)
x11=concatenate([x8,x10],axis=2)
x12=Dropout(dropout_rate)(x11)
x12=BatchNormalization()(x12)
x13=Dense(80,activation='relu',
       kernel_regularizer=regularizers.l2(l2_rate))(x12)
x14=concatenate([x11,x13],axis=2)
x15=Dropout(dropout_rate_model)(x14)
x15=BatchNormalization()(x15)
x16=Dense(800,activation='relu',
       kernel_regularizer=regularizers.l2(l2_rate))(x15)
x17=Dropout(dropout_rate)(x16)
x17=BatchNormalization()(x17)
x18=Dense(200,activation='relu',
       kernel_regularizer=regularizers.l2(l2_rate))(x17)
x19=Dropout(dropout_rate)(x18)
x19=BatchNormalization()(x19)
x20=Dense(100,activation='relu',
       kernel_regularizer=regularizers.l2(l2_rate))(x19)
x21=Dropout(dropout_rate)(x20)
x21=BatchNormalization()(x21)
outputs=Dense(num_classes,activation='softmax')(x21)
model=Model(input_data,outputs)

print('model generated')

print('begin training')
X_train,X_test, y_train, y_test =train_test_split(train_data_1200,true_label,test_size=validation_split, random_state=0)
print('data splited')
model.compile(loss='categorical_crossentropy',
              #loss=my_logloss,
              optimizer=Adam(0.001),#(lr=lr_schedule(0)),#(lr=l0.001),
              metrics=['accuracy'])
model.summary()
model_type='DNN_1200'
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'male_age_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             #monitor='val_acc',
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True)

lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=1e-6)

#callbacks = [checkpoint, lr_reducer,lr_scheduler]
callbacks = [checkpoint, lr_reducer]

train = model.fit(X_train,y_train,epochs=epochs,
                  batch_size=batch_size,validation_data=(X_test,y_test),
                  verbose=1, callbacks=callbacks)
scores=model.evaluate(X_test,y_test,batch_size=batch_size,verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

y_pred_dnn22=model.predict(X_test)
y_pred_dnn22=y_pred_dnn22.reshape(len(y_pred_dnn22),22)
y_test=y_test.reshape(len(y_test),22)
result=0
log_loss_score=0
for i in range(len(y_pred_dnn22)):
    for j in range(22):
        yp=y_pred_dnn22[i][j]
        if yp==0.0:
            yp=1e-6
        result+=(math.log(yp))*(y_test[i][j])
    log_loss_score+=result
    result=0
log_loss_score=-log_loss_score/len(y_pred_dnn22)
print('log_loss_score=%f' %log_loss_score)


model generated
begin training
data splited
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 1, 1200)      0                                            
__________________________________________________________________________________________________
dense_43 (Dense)                (None, 1, 1200)      1441200     input_8[0][0]                    
__________________________________________________________________________________________________
concatenate_36 (Concatenate)    (None, 1, 2400)      0           input_8[0][0]                    
                                                                 dense_43[0][0]                   
__________________________________________________________________________________________________
dropout_36 (Dropout)            (None, 1, 2400)      0           

In [16]:
import numpy as np
from keras.layers import Dense,Flatten,Input,Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPool1D
from keras.layers.merge import concatenate
from keras.models import Model
from keras.models import load_model
from keras.optimizers import Adam
import keras
import math
import os
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from keras import backend as k_backend
from keras import regularizers
from sklearn import preprocessing
from sklearn.decomposition import PCA
from keras.layers import BatchNormalization


def my_logloss(y_true,y_pred):
    return -k_backend.sum(y_true*k_backend.log(y_pred),axis=-1)
train_data_1200=np.load('new_feature_1200.npy')
train_data_1200=train_data_1200-train_data_1200.mean()
data_len=len(train_data_1200)
true_label=np.load('true_label.npy')
true_label=true_label.reshape(data_len,1,22)
validation_split=0.1
batch_size=2048*8
epochs=1000
num_classes=22

#dropout_rate=0.8
#dropout_rate_model=0.8
#l2_rate=0.0001
dropout_rate=0.9
#dropout_rate_model=0
l2_rate=0.01

#haha，pencil model
input_data=Input(shape=(1,train_data_1200.shape[2]),dtype='float')
x1=Dense(1200,activation='relu')(input_data)
x2=concatenate([input_data,x1],axis=2)
x3=Dropout(dropout_rate)(x2)
x3=BatchNormalization()(x3)
x4=Dense(600,activation='relu')(x3)
x5=concatenate([x2,x4],axis=2)
x6=Dropout(dropout_rate)(x5)
x6=BatchNormalization()(x6)
x7=Dense(300,activation='relu')(x6)
x8=concatenate([x5,x7],axis=2)
x9=Dropout(dropout_rate)(x8)
x9=BatchNormalization()(x9)
x10=Dense(150,activation='relu')(x9)
x11=concatenate([x8,x10],axis=2)
x12=Dropout(dropout_rate)(x11)
x12=BatchNormalization()(x12)
x13=Dense(80,activation='relu')(x12)
x14=concatenate([x11,x13],axis=2)
x15=Dropout(dropout_rate)(x14)
x15=BatchNormalization()(x15)
x16=Dense(800,activation='relu')(x15)
x17=Dropout(dropout_rate)(x16)
x17=BatchNormalization()(x17)
x18=Dense(200,activation='relu')(x17)
x19=Dropout(dropout_rate)(x18)
x19=BatchNormalization()(x19)
x20=Dense(100,activation='relu')(x19)
x21=Dropout(dropout_rate)(x20)
x21=BatchNormalization()(x21)
outputs=Dense(num_classes,activation='softmax')(x21)
model=Model(input_data,outputs)

print('model generated')

print('begin training')
X_train,X_test, y_train, y_test =train_test_split(train_data_1200,true_label,test_size=validation_split, random_state=0)
print('data splited')
model.compile(loss='categorical_crossentropy',
              #loss=my_logloss,
              optimizer=Adam(0.001),#(lr=lr_schedule(0)),#(lr=l0.001),
              metrics=['accuracy'])
model.summary()
model_type='DNN_1200'
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'male_age_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             #monitor='val_acc',
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True)

lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=1e-6)

#callbacks = [checkpoint, lr_reducer,lr_scheduler]
callbacks = [checkpoint, lr_reducer]

train = model.fit(X_train,y_train,epochs=epochs,
                  batch_size=batch_size,validation_data=(X_test,y_test),
                  verbose=1, callbacks=callbacks)
scores=model.evaluate(X_test,y_test,batch_size=batch_size,verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

y_pred_dnn22=model.predict(X_test)
y_pred_dnn22=y_pred_dnn22.reshape(len(y_pred_dnn22),22)
y_test=y_test.reshape(len(y_test),22)
result=0
log_loss_score=0
for i in range(len(y_pred_dnn22)):
    for j in range(22):
        yp=y_pred_dnn22[i][j]
        if yp==0.0:
            yp=1e-6
        result+=(math.log(yp))*(y_test[i][j])
    log_loss_score+=result
    result=0
log_loss_score=-log_loss_score/len(y_pred_dnn22)
print('log_loss_score=%f' %log_loss_score)


model generated
begin training
data splited
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 1, 1200)      0                                            
__________________________________________________________________________________________________
dense_70 (Dense)                (None, 1, 1200)      1441200     input_11[0][0]                   
__________________________________________________________________________________________________
concatenate_51 (Concatenate)    (None, 1, 2400)      0           input_11[0][0]                   
                                                                 dense_70[0][0]                   
__________________________________________________________________________________________________
dropout_60 (Dropout)            (None, 1, 2400)      0           

KeyboardInterrupt: ignored

In [11]:
!ls -l Demo

total 3404528
-rw-r--r-- 1 501 staff    3644867 Jul  3 09:34 deviceid_brand.tsv
-rw-r--r-- 1 501 staff 3451710410 Jul  3 09:36 deviceid_package_start_close.tsv
-rw-r--r-- 1 501 staff   27668058 Jul  3 09:35 deviceid_packages.tsv
-rw-r--r-- 1 501 staff     749991 Jul  3 09:37 deviceid_test.tsv
-rw-r--r-- 1 501 staff    1854703 Jul  3 09:37 deviceid_train.tsv
-rw-r--r-- 1 501 staff     582238 Jul  3 09:36 package_label.tsv
-rw-r--r-- 1 501 staff       9194 Jul  3 10:38 性别年龄对照表.xlsx


In [12]:
#generate train_data by traim_label
import pandas as pd
import numpy as np
import gc
import time

deviceid_packages_data = pd.read_csv('./Demo/deviceid_packages.tsv',header=None,sep='\t')
deviceid_package_start_close_data = pd.read_csv('./Demo/deviceid_package_start_close.tsv',header=None,sep='\t')
label_data = pd.read_csv('./Demo/deviceid_train.tsv',header=None,sep='\t')
print('data loaded')

train_label=[]
deviceid_total=[]
app_all_encode=[]
app_all_time=[]

listZero = [0]
listVal = listZero * 128

#generate  deviceid_total
for num in range (len(deviceid_packages_data.iloc[:,0])):
    deviceid_total.append(deviceid_packages_data.iloc[num,0])

counter=0
for train_id in label_data.iloc[:,0]:
    begin_time=time.time()
    aa=train_id   
    bb=deviceid_packages_data.iloc[:,0]
    cc=bb[(bb.isin([aa]))]
    seek_deviceid_packages_index=[int(qq) for qq in cc.index]
    app_list=deviceid_packages_data.iloc[seek_deviceid_packages_index[0],1]
    app_list=app_list.split(',')
    app_list_encode=[]
    app_time_encode=[]
    for j in range (len(app_list)):
        sample_bin=bin(int(app_list[j],16))[2:] #del 0b
        sample_single=[int(bit) for bit in sample_bin]
        for k in range((128-len(sample_single))):
            sample_single.insert(k,0)        
        app_list_encode.append(sample_single)
        
        aa=train_id
        bb=deviceid_package_start_close_data.iloc[:,0]
        cc=bb[(bb.isin([aa]))]
        seek_start_close_index=[int(qq) for qq in cc.index]
        
        aa=app_list[j]
        bb=deviceid_package_start_close_data.iloc[seek_start_close_index,1]
        cc=bb[(bb.isin([aa]))]
        seek_time_index=[int(qq) for qq in cc.index]
        app_temp=deviceid_package_start_close_data.iloc[seek_time_index,3]-deviceid_package_start_close_data.iloc[seek_time_index,2]
        app_time=sum(app_temp)
        app_time_bin=bin(app_time)[2:]
        app_time_single=[int(bit) for bit in app_time_bin]
        for k in range((128-len(app_time_single))):
            app_time_single.insert(k,0)        
        app_time_encode.append(app_time_single)
        
    encode_line=len(app_list_encode)
    if encode_line<174:
        for l in range(174-encode_line):
            app_list_encode.append(listVal)
    app_all_encode.append(app_list_encode)
    
    time_line=len(app_time_encode)
    if time_line<174:
        for l in range(174-time_line):
            app_time_encode.append(listVal)
    app_all_time.append(app_time_encode)
    counter+=1
    if counter%100==0:
        print(counter)
        stop_time=time.time()
        time_elapsed=stop_time-begin_time
        print('time of 100 sample=',time_elapsed)

train_data_channel_encode=np.asarray(app_all_encode,dtype='uint8')
train_data_channel_time=np.asarray(app_all_time,dtype='uint8')
np.save('train_data_channel_encode.npy',train_data_channel_encode)
np.save('train_data_channel_time.npy',train_data_channel_time)
print('data saved')

data loaded
100
('time of 100 sample=', 1.356368064880371)
200
('time of 100 sample=', 4.083027124404907)
300
('time of 100 sample=', 3.2462668418884277)
400
('time of 100 sample=', 4.501523017883301)
500
('time of 100 sample=', 5.999797105789185)
600
('time of 100 sample=', 6.333871126174927)
700
('time of 100 sample=', 2.0794270038604736)


KeyboardInterrupt: ignored